In [2]:
from sklearn.naive_bayes import MultinomialNB

In [1]:
#convert list of mini dict to list of full list
def convert(list_of_word_fre, vocab_set_len):
    list_of_vector = []
    print ('Converting list of mini dict to list of full list')
    for words_fre in list_of_word_fre:
        one_vector = [0 for i in range(vocab_set_len)]
        for word_id, value in words_fre.items():
            one_vector[word_id] += value
        list_of_vector.append(one_vector)
    print ('Finished converting list of mini dict to list of full list')
    return list_of_vector

In [4]:
#set labels
def set_label(num_neg = 12500, num_pos = 12500,first = 'neg', second = 'pos'):
    labels = []
    for i in range(num_neg):
        labels.append(first)
    for i in range(num_pos):
        labels.append(second)
    return labels

In [6]:
#train dataset
def train_for_clf(train_data, train_labels):
    clf = MultinomialNB()
    print ('Training for clfer')
    clf.fit(train_data, train_labels)
    print ('Finished training')
    return clf

In [4]:
#non-libraries
def sub_train(list_of_word_fre, training_vocab_size):
    pretrain_dict = {}
    for dict_word_fre in list_of_word_fre:
        for index, fre in dict_word_fre.items():
            if index not in pretrain_dict.keys():
                pretrain_dict[index] = fre
            else:
                pretrain_dict[index] += fre
    for index in range (training_vocab_size):
        if index not in pretrain_dict.keys():
            pretrain_dict[index] = 0
    N = 0
    for value in pretrain_dict.values():
        N += value
    N += training_vocab_size # Laplace smoothing
    for index, fre in pretrain_dict.items():
        pretrain_dict[index] = (pretrain_dict[index] + 1) / N # Laplace smoothing
    return pretrain_dict

In [5]:
def train(list_of_word_fre, training_vocab_size, neg_vocab_size):
    print ('Training neg data')
    neg_pretrain_dict = sub_train(list_of_word_fre[:neg_vocab_size],training_vocab_size)
    print ('Finish training neg data')
    print ('Training pos data')
    pos_pretrain_dict = sub_train(list_of_word_fre[neg_vocab_size:],training_vocab_size)
    print ('Finish training pos data')
    return neg_pretrain_dict, pos_pretrain_dict

In [6]:
def predict(neg_pretrain_dict, pos_pretrain_dict, test_list_of_word_fre, test_label):
    import math
    predict_label = []
    #count for neg
    for dict_word_fre in test_list_of_word_fre:
        neg_predict = 0
        pos_predict = 0
        for index, fre in dict_word_fre.items():
            neg_predict += math.log(neg_pretrain_dict[index])*fre
            pos_predict += math.log(pos_pretrain_dict[index])*fre
        if neg_predict > pos_predict:
            predict_label.append('neg')
        else:
            predict_label.append('pos')
    count = 0
    total_label = len(test_label)
    for i in range(total_label):
        if predict_label[i] == test_label[i]:
            count += 1
    accuracy = count / total_label
    return accuracy
        
        